In [1]:
library(data.table)

In [2]:
## Using a table of article assessments and views, build tables
## (matrices) that shows the number of dissonant articles per
## assessment category based on sorting by popularity.
##
## The underlying assumption is that in an ideal system with a limited
## and fixed amount of resources (in other words, popularity and high quality
## artefacts does not increase the amount of resources in the system),
## popularity ranking and assessment class follow a 1-to-1 relationship.
## We can therefore sort by popularity and group articles that way
## because work will be prioritised by popularity.

## DATA ASSUMPTION: views_with_redirects from resolve-redirects.R
## is loaded into memory.

## 3: build a 2x2 matrix of assessment classes and popularity classes
## 

## Assessment classes in ascending order of quality.

In [3]:
assessment_classes = c('E', 'D', 'C', 'B', 'A');

In [ ]:
quality_prediction_and_page_views <- read.table("../../results/sql_queries/entity_views_and_aggregated_revisions/entity_views_and_aggregated_revisions_and_quality_scoring_20170701.tsv", header=FALSE, sep="\t")

In [ ]:
quality_prediction_and_page_views <- data.table(quality_prediction_and_page_views)

In [ ]:
colnames(quality_prediction_and_page_views) <- c('entity_id','number_of_revisions', 'page_views', 'prediction', 'ordinal_score')

In [ ]:
summary(quality_prediction_and_page_views)

In [ ]:
## 0: calculate number of articles in each assessment class
n_per_class = quality_prediction_and_page_views[, list(narticles=sum(.N)), by='prediction']

In [ ]:
setkey(n_per_class, prediction);
## NOTE: setkey allows us to do n_per_class['GA']$narticles to get counts

In [ ]:
## 1: order articles by popularity
articles_by_pop = quality_prediction_and_page_views[order(quality_prediction_and_page_views$page_views)][,list(entity_id, prediction, page_views)];



In [ ]:
## 2: assign popularity assessment class based on rank
##   (buckets based on number of articles in each class)
articles_by_pop[, pop_class := ''];
articles_by_pop[, seqNum := seq_len(nrow(articles_by_pop))];

In [ ]:
assign_pop_class = function(dataset, classes, class_n) {
  ## Based on the per-class number of articles in class_n
  ## assign popularity based on classes to dataset.
  prev_idx = 0;
  for(rating in classes) {
    start_idx = prev_idx + 1;
    end_idx = start_idx + class_n[prediction == rating]$narticles;
    print(paste('start_idx =', start_idx, ', end_idx = ', end_idx));
    dataset[seqNum >= start_idx & seqNum <= end_idx, pop_class := rating];
    prev_idx = end_idx -1;
  }
  dataset;
}

In [ ]:
articles_by_pop = assign_pop_class(articles_by_pop,
  assessment_classes, n_per_class);

In [ ]:
create_dissonance_matrix = function(articledata, classes) {
  d_mtrx = matrix(0, nrow=length(classes), ncol=length(classes));
  rownames(d_mtrx) = classes;
  colnames(d_mtrx) = classes;

  for(real_rating in classes) {
    for(pop_rating in classes) {
      d_mtrx[real_rating, pop_rating] = length(articledata[prediction == real_rating & pop_class == pop_rating]$entity_id);
    }
  }
  d_mtrx;
}


In [ ]:
## Based on direct hits to articles:
create_dissonance_matrix(articles_by_pop, assessment_classes)

In [ ]:
dissonance_matrix = create_dissonance_matrix(articles_by_pop,
  assessment_classes);

In [ ]:
prediction_e_pop_class_a <- merge(articles_by_pop[prediction == 'E' & pop_class == 'A'],quality_prediction_and_page_views, by='entity_id')[, c("entity_id","page_views.x", "number_of_revisions")]

In [ ]:
head(prediction_e_pop_class_a)

In [ ]:
## Q: why do I get _two_ pageid columns?  Solution is to do the selection
## on the joined table, not as a select _in_ the join.

## Dissonance matrix proportions by row (..., 1) and column (..., 2)
## rounded to 1 decimal places.

In [ ]:
round(100*prop.table(dissonance_matrix, 1), 1);

In [ ]:
round(100*prop.table(dissonance_matrix, 2), 1);

In [ ]:
## Let's write the stubs out to a file
# write.table(views_with_redirects[articles_by_pop[assessment_class == 'Stub' & pop_class == 'FA']][,list(pageid, pagetitle, loghits, logmeanhits)],
#            '../datasets/assessments/stubs-fa-by-rank.txt');

# Dissonance Measures (was seperate file)

In [ ]:
## Various ways of measuring dissonance.

## DATA ASSUMPTION: articles_by_pop from build-dissonance-table.R
## is loaded into memory.

## None/Moderate/High measure of dissonance

In [ ]:
articles_by_pop[, pop_class := ordered(pop_class, assessment_classes)];

In [ ]:
dissonance_metric = c('High negative', 'Moderate negative',
  'None', 'Moderate positive', 'High positive');

In [ ]:
articles_by_pop[, dissonance := factor(NA, dissonance_metric)];

In [ ]:
## NOTE: because pop_class is of class ordered, we can use
##       expressions like "pop_class < 'C'" as expected

In [ ]:
## A: None if A, Moderate if A, High elsewhere
articles_by_pop[prediction == 'A' & pop_class <= 'C',
                dissonance := 'High negative'];
articles_by_pop[prediction == 'A' & pop_class == 'B',
                dissonance := 'Moderate negative'];
articles_by_pop[prediction == 'A' & pop_class == 'A',
                dissonance := 'None'];


In [ ]:
## B: 
articles_by_pop[prediction == 'B' & pop_class <= 'D',
                dissonance := 'High negative'];
articles_by_pop[prediction == 'B' & pop_class == 'C',
                dissonance := 'Moderate negative'];
articles_by_pop[prediction == 'B' & pop_class == 'B',
                dissonance := 'None'];
articles_by_pop[prediction == 'B' & pop_class == 'A',
                dissonance := 'Moderate positive'];

In [ ]:
## C: 
articles_by_pop[prediction == 'C' & pop_class == 'E',
                dissonance := 'High negative'];
articles_by_pop[prediction == 'C' & pop_class == 'D',
                dissonance := 'Moderate negative'];
articles_by_pop[prediction == 'C' & pop_class == 'C',
                dissonance := 'None'];
articles_by_pop[prediction == 'C' & pop_class == 'B',
                dissonance := 'Moderate positive'];
articles_by_pop[prediction == 'C' & pop_class == 'A',
                dissonance := 'High positive'];

In [ ]:
## D
articles_by_pop[prediction == 'D' & pop_class == 'E',
                dissonance := 'Moderate negative'];
articles_by_pop[prediction == 'D' & pop_class == 'D',
                dissonance := 'None'];
articles_by_pop[prediction == 'D' & pop_class == 'C',
                dissonance := 'Moderate positive'];
articles_by_pop[prediction == 'D' & pop_class >= 'B',
                dissonance := 'High positive'];

In [ ]:
## E
articles_by_pop[prediction == 'E' & pop_class == 'E',
                dissonance := 'None'];
articles_by_pop[prediction == 'E' & pop_class == 'D',
                dissonance := 'Moderate positive'];
articles_by_pop[prediction == 'E' & pop_class >= 'C',
                dissonance := 'High positive'];

In [ ]:
## Build a matrix where columns are the metric and rows are classes
create_alt_diss_matrix = function(articledata, metric, classes) {
  d_mtrx = matrix(0, nrow=length(classes), ncol=length(metric));
  rownames(d_mtrx) = classes;
  colnames(d_mtrx) = metric;

  ## NOTE: R matrix values are [row,col] dimensions
  for(real_rating in classes) {
    for(diss_rating in metric) {
      d_mtrx[real_rating, diss_rating] = length(articledata[prediction == real_rating & dissonance == diss_rating]$entity_id);
    }
  }
  d_mtrx;
}

alternative_dissonance_matrix.1 = create_alt_diss_matrix(articles_by_pop,
  dissonance_metric, assessment_classes);




In [ ]:
## Normalise by row
round(100*prop.table(alternative_dissonance_matrix.1, 1), 1);




In [ ]:
## Number of dissonant views per assessment class and amount of dissonance
articles_by_pop[, list(dissonant_views=sum(page_views)), by=list(prediction, dissonance)];



In [ ]:
## Calculations of total number of dissonant views per dissonance
articles_by_pop[, list(dissonant_views=sum(page_views)), by=list(dissonance)];

In [ ]:
articles_by_pop[,sum(as.numeric(page_views))];

In [ ]:
## Proportions
100*65938379920/545180810059;
100*125047198/545180810059;
100*6713682043/545180810059;
100*120523625541/545180810059;
100*351880075357/545180810059;

In [ ]:
# 87% of views are high positive

In [ ]:
100*(articles_by_pop[, list(dissonant_views=sum(page_views)), by=list(dissonance)][1][,c('dissonant_views')]/articles_by_pop[,sum(as.numeric(page_views))])

In [ ]:
100*(articles_by_pop[, list(dissonant_views=sum(page_views)), by=list(dissonance)][2][,c('dissonant_views')]/articles_by_pop[,sum(as.numeric(page_views))])

In [ ]:
100*(articles_by_pop[, list(dissonant_views=sum(page_views)), by=list(dissonance)][3][,c('dissonant_views')]/articles_by_pop[,sum(as.numeric(page_views))])

In [ ]:
100*(articles_by_pop[, list(dissonant_views=sum(page_views)), by=list(dissonance)][4][,c('dissonant_views')]/articles_by_pop[,sum(as.numeric(page_views))])

In [ ]:
100*(articles_by_pop[, list(dissonant_views=sum(page_views)), by=list(dissonance)][5][,c('dissonant_views')]/articles_by_pop[,sum(as.numeric(page_views))])